# Content-Based Movie Recommendation

Dataset Source:

https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv("../data/wiki_movie_plots_deduped.csv")
df.shape

(34886, 8)

In [4]:
df.sample(5)

Release Year                   Title Origin/Ethnicity      Director  \
5307           1951           The Big Night         American  Joseph Losey   
3302           1942           Flying Tigers         American  David Miller   
16548          2013                     Her         American   Spike Jonze   
4283           1946      The Mask of Diijon         American   Lew Landers   
29973          1992  Onna Irukka Kathukanum            Tamil     V. Shekar   

                                                    Cast      Genre  \
5307              John Drew Barrymore, Dorothy Comingore  film noir   
3302                                          John Wayne        war   
16548  Joaquin Phoenix, Amy Adams, Rooney Mara, Scarl...    romance   
4283                    Erich von Stroheim, Jeanne Bates      drama   
29973                       Sivakumar, Jeeva, Goundamani    unknown   

                                               Wiki Page  \
5307   https://en.wikipedia.org/wiki/The_Big_Night_(1...   
3302   https://en.wikipedia.org/wiki/Flying_Tigers_(f...   
16548           https://en.wikipedia.org/wiki/Her_(film)   
4283    https://en.wikipedia.org/wiki/The_Mask_of_Diijon   
29973  https://en.wikipedia.org/wiki/Onna_Irukka_Kath...   

                                                    Plot  
5307   On his teenaged son Georgie's birthday, Andy L...  
3302   Jim Gordon (John Wayne in his first war film) ...  
16548  In a near future Los Angeles, Theodore Twombly...  
4283   Diijon, a tired magician, gives up his act to ...  
29973  The story begins with a group of government of...

## Preprocessing

### Converting to String

In [5]:
df.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [6]:
df = df.astype(str)
df.dtypes

Release Year        object
Title               object
Origin/Ethnicity    object
Director            object
Cast                object
Genre               object
Wiki Page           object
Plot                object
dtype: object

### Data Cleaning

Action Items:

* Lower-Case the whole data frame
* Director: Removing 'Director:' and 'Cast:'
* Director, Cast: Removing '\r\n', '\n' and '\r'

* Genre: Replacing '/' with Space
* Director, Cast, Genre: Removing 'Uknonwn' and 'Nan'

* Director: Separating Directors and Actors names
* Director, Cast: Checking if the names are separated with ' and ', ' & '

* Director, Cast: Merging the first names and last names together
* Director, Cast: Adding the words of 'Director' and 'Actor' as prefix

* Plot: Removing English Stopwords
* Doc: Removing special characters

In [7]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-7-614eaff662ae>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [8]:
df["Director"] = df["Director"].str.replace("director:", "", regex=False)
df["Director"] = df["Director"].str.replace("cast:", "", regex=False)

df["Director"] = df["Director"].str.replace("\r\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\r\n", " ", regex=False)

df["Director"] = df["Director"].str.replace("\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\n", " ", regex=False)

df["Genre"] = df["Genre"].str.replace("/", " ", regex=False)

df["Director"] = df["Director"].str.replace("unknown", "", regex=False)
df["Cast"] = df["Cast"].str.replace("unknown", "", regex=False)
df["Genre"] = df["Genre"].str.replace("unknown", "", regex=False)

df["Director"] = df["Director"].str.replace("nan", "", regex=False)
df["Cast"] = df["Cast"].str.replace("nan", "", regex=False)
df["Genre"] = df["Genre"].str.replace("nan", "", regex=False)


df["Director"] = df["Director"].str.replace(" and ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" and ", ",", regex=False)
df["Director"] = df["Director"].str.replace(" & ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" & ", ",", regex=False)


df["Director"] = df["Director"].str.replace(" ", "", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", "", regex=False)

df["Director"] = df["Director"].str.replace(",", " ", regex=False)
df["Cast"] = df["Cast"].str.replace(",", " ", regex=False)


In [9]:
df["Director"] = np.where(df["Director"].str.len() > 0,
                          'director' + df["Director"],
                          df["Director"])

df["Cast"] = np.where(df["Cast"].str.len() > 0,
                      'actor' + df["Cast"],
                      df["Cast"])

df["Director"] = df["Director"].str.replace(" ", " director", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", " actor", regex=False)


In [10]:
df.sample(5)

Release Year            Title Origin/Ethnicity  \
26026         2000           baaghi        bollywood   
27449         2012            rambo          kannada   
30429         2001            kutty            tamil   
15457         2008  the house bunny         american   
13681         2000       lost souls         american   

                         Director  \
26026    directorrajeshkumarsingh   
27449        directorm.s.sreenath   
30429  directorjanakivishwanathan   
15457            directorfredwolf   
13681      directorjanuszkamiński   

                                                    Cast   Genre  \
26026                 actorsunildutt actormanishakoirala  action   
27449  actorsharan actormadhuri actordeepika actorshr...  comedy   
30429  actorp.shwetha actorramesharavind actoreaswarirao           
15457  actorannafaris actoremmastone actorcolinhanks ...  comedy   
13681                   actorwinonaryder actorbenchaplin  horror   

                                              Wiki Page  \
26026  https://en.wikipedia.org/wiki/baaghi_(2000_film)   
27449   https://en.wikipedia.org/wiki/rambo_(2012_film)   
30429   https://en.wikipedia.org/wiki/kutty_(2001_film)   
15457     https://en.wikipedia.org/wiki/the_house_bunny   
13681   https://en.wikipedia.org/wiki/lost_souls_(film)   

                                                    Plot  
26026  raja (sanjay dutt) is in love with a night clu...  
27449  ‘rambo’ is a film that revolves around a trick...  
30429  the film revolves around a young girl who is f...  
15457  shelley darlington (anna faris) is an aspiring...  
13681  a small group of fervent roman catholics belie...

### Merging the document

In [11]:
column_weights = {"Release Year": 10,
                  "Title": 1,
                  "Origin/Ethnicity": 5,
                  "Director": 5,
                  "Cast": 1,
                  "Genre": 10,
                  "Plot": 1}

df["doc"] = ""

for col in column_weights.keys():
    df["doc"] += column_weights[col] * (df[col] + ' ')

df.head()

Release Year                             Title Origin/Ethnicity  \
0         1901            kansas saloon smashers         american   
1         1901     love by the light of the moon         american   
2         1901           the martyred presidents         american   
3         1901  terrible teddy, the grizzly king         american   
4         1902            jack and the beanstalk         american   

                                        Director Cast Genre  \
0                                                             
1                                                             
2                                                             
3                                                             
4  directorgeorges.fleming directoredwins.porter              

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/kansas_saloon_sm...   
1  https://en.wikipedia.org/wiki/love_by_the_ligh...   
2  https://en.wikipedia.org/wiki/the_martyred_pre...   
3  https://en.wikipedia.org/wiki/terrible_teddy,_...   
4  https://en.wikipedia.org/wiki/jack_and_the_bea...   

                                                Plot  \
0  a bartender is working at a saloon, serving dr...   
1  the moon, painted with a smiling face hangs ov...   
2  the film, just over a minute long, is composed...   
3  lasting just 61 seconds and consisting of two ...   
4  the earliest known adaptation of the classic f...   

                                                 doc  
0  1901 1901 1901 1901 1901 1901 1901 1901 1901 1...  
1  1901 1901 1901 1901 1901 1901 1901 1901 1901 1...  
2  1901 1901 1901 1901 1901 1901 1901 1901 1901 1...  
3  1901 1901 1901 1901 1901 1901 1901 1901 1901 1...  
4  1902 1902 1902 1902 1902 1902 1902 1902 1902 1...

### Removing Special Characters

In [31]:
df["doc"] = df["doc"].str.replace("[^a-z 0-9]+", "", regex=True)

In [32]:
pd.set_option('display.max_colwidth', None)
df[["doc"]].sample(10)

doc
9436                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        1977 1977 1977 1977 1977 1977 1977 1977 1977 1977 viva knievel american american american american american directorgordondouglas directorgordondouglas directorgordondouglas directorgordondouglas directorgordondouglas actorevelknievel actorlaurenhutton actorgenekelly action biography action biography action biography action biography action biography action biography action biography action biography action biography action biography daredevil motorcycle rider evel knievel stars as himself in this fictional story the film opens with knievel sneaking into an orphanage late at night to deliver presents evel knievel action figures one of the boys casts away his crutches telling knievel that hell walk after his accident just as knievel hadknievel then prepares for another of his stunt jumps we are introduced to his alcoholic mechanic will atkins gene kelly who was a former stunt rider himself before his wife died driving him to drink while signing autographs knievel is ambushed by feminist photojournalist kate morgan lauren hutton who has been sent to photograph the jump if knievel is killed it will be a great storyas it happens evel does crash while attempting the stunt and though badly injured survives he berates morgan announces his retirement and is taken to the hospitalwhile rehabilitating knievel resists all attempts to get back on the horse including those from jessie marjoe gortner a former protg with mysterious backers who want evel to do a jump in mexico eventually though knievel relents and agreesa subplot develops when wills estranged son tommy shows up from boarding school and asks to join the tour will who is reminded of his dead wife is cold to tommy leaving knievel to show the boy kindness likewise kate reappears apologetic for her previous motives and now wishes that he will never stop jumpingmeanwhile jessies benefactor is revealed drug lord stanley millard leslie nielsen millard without jessies knowledge plans to cause a fatal accident during the jump he will then have knievels body transported back to america in an exact duplicate of the tour trailer but one that has a massive supply of drugs hidden in the wallswill however stumbles onto the plot is drugged and sent to a psychiatric ward under the control of the corrupt ralph thompson dabney coleman to prevent him from spilling the beans evel sneaks into the ward late at night when will has dried out but all will can remember is that someone knocked him out knievel leaves him there to keep whoever is behind the plot in the darkas knievel prepares for the jump down a massive ramp and over a fire pit jessiehopped up on drugsconfronts evel claiming that he will prove who the best jumper is jessie knocks evel out and dresses in knievels signature red white and blue outfit jessie then successfully makes the jump however the bike has been sabotaged and he is killed as he lands footage from a real knievel crash was used while the body is taken away for the drug smuggling plot evel wakes up gets on another bike and goes to free willafter breaking out of the psych ward the two find the mockup trailer in which by an amazing coincidence both tommy and kate have been taken hostage pursuing the truck will and evel decide to split up will will disable the semi evel will lead off the guntoting drug lords riding guard in another carat the end of several extended chase scenes the drug lords are defeated will and his son are reunited and kate has fallen head ov

### Removing Stopwords

In [15]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords

In [16]:
stops = stopwords.words('english')
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

df['doc_clean'] = df['doc'].map(lambda s:preprocess(s)) 

In [18]:
df[["doc", 'doc_clean']].sample(10)

doc  \
6122                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## TF-IDF

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["doc_clean"])
column_names = vectorizer.get_feature_names_out()

df_tf_idf = pd.DataFrame(X.toarray(), columns=column_names)
df_tf_idf.shape

(34886, 251188)

## Cosine Similarity

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

df_cos_sim = pd.DataFrame(cosine_similarity(df_tf_idf, dense_output=True))
df_cos_sim.shape

/opt/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


(34886, 34886)

### Converting Cosine Similarity Dataframe to Top-K Items

In [34]:
import warnings

# hide pandas warning messages
warnings.filterwarnings('ignore')

In [35]:
from tqdm.notebook import tqdm

K = 10

df_top_k = pd.DataFrame()
movie_indices = df_cos_sim.columns

for col in tqdm(movie_indices):
    df_test = df_cos_sim[[col]].sort_values(by=[col], ascending=False).head(K+1).copy()

    record = []
    for index, row in df_test.iterrows():
        if index != col:
            item = [int(index), float(row[col])]
            record.append(item)
        if len(record) == K:
            break

    df_top_k[col] = record

df_top_k.shape

  0%|          | 0/34886 [00:00<?, ?it/s]

(10, 34886)

In [40]:
# Transpose
df_top_k = df_top_k.T

df_top_k.sample(10)

0                             1  \
26025   [26504, 0.3620489229730903]   [28718, 0.3252340008773924]   
9008     [9004, 0.5122274405830288]  [33037, 0.40420383857958814]   
32847   [32846, 0.8551423322970173]   [7035, 0.47972423586536644]   
13856   [21513, 0.3480367039522531]   [4134, 0.33309619583649946]   
18732   [18722, 0.4554976874767943]  [34527, 0.42096436605275983]   
7719     [7771, 0.6928682794762017]   [19692, 0.6441729182501121]   
9797   [25175, 0.33713114014245027]   [25169, 0.3291603321478832]   
3866    [20720, 0.4761706754356434]   [18096, 0.4597195837196465]   
4948     [4981, 0.5577362848276128]    [4944, 0.5479257338775796]   
31532   [5949, 0.49527548524900766]    [5864, 0.4622274547380769]   

                                  2                             3  \
26025   [28632, 0.3051069020176202]  [26299, 0.28905304730763376]   
9008    [9011, 0.36707442778749066]  [17407, 0.35376842243076123]   
32847   [7086, 0.46971844215945846]   [6980, 0.46373064326102514]   
13856   [6491, 0.31260049713129345]   [4851, 0.31247903461471976]   
18732   [34295, 0.3973296482392984]  [34521, 0.39487003463827286]   
7719    [19669, 0.5575402790721555]   [32893, 0.5300184873899625]   
9797    [25177, 0.3245161038619608]   [25174, 0.3220402493609471]   
3866   [18097, 0.44350283022139225]   [8705, 0.38917010967286564]   
4948    [18869, 0.5398801620871491]    [4847, 0.5381642563952086]   
31532    [5915, 0.4449189407699913]     [5923, 0.442325995196076]   

                                  4                             5  \
26025  [23910, 0.27672501700917873]   [27536, 0.2139407924188466]   
9008    [9018, 0.35182061629607597]    [9098, 0.3464159686356937]   
32847   [7017, 0.46362807486332214]   [19390, 0.4607687751089281]   
13856   [9520, 0.29941393847889713]    [8813, 0.2958359293688111]   
18732   [4478, 0.39245923280639516]  [34297, 0.38878224141035606]   
7719    [19694, 0.4737045688335866]   [7733, 0.44998808656901274]   
9797     [9798, 0.3190776894344768]   [9853, 0.31744055351477807]   
3866     [7478, 0.3773605302515406]   [17819, 0.3592525187653775]   
4948     [5002, 0.5330969199825556]     [4893, 0.528909395976042]   
31532    [31531, 0.431939880581105]   [19106, 0.4276563220369206]   

                                  6                             7  \
26025  [13906, 0.20152185292688773]  [26049, 0.16824506757415866]   
9008     [9045, 0.3461904367377711]   [33031, 0.3441738821188626]   
32847    [19375, 0.460080621617711]   [6974, 0.45951762301714283]   
13856   [10282, 0.2935366295258927]  [12376, 0.29226806485317436]   
18732   [4346, 0.38758392085708215]   [24706, 0.3826056606863259]   
7719    [34319, 0.4456154623896174]   [7758, 0.44543937140590484]   
9797     [25163, 0.311862764952367]   [17502, 0.3101844778511574]   
3866    [10885, 0.3522126009760382]  [18676, 0.33897679720172597]   
4948    [18857, 0.5235814248231784]     [5032, 0.522102456249883]   
31532   [5854, 0.42712418685233755]  [19091, 0.42665424194272517]   

                                  8                             9  
26025   [26654, 0.1561447509224819]  [26896, 0.15468184455730213]  
9008     [9005, 0.3420256605247796]   [9064, 0.34175097043130215]  
32847  [19382, 0.45810861837129235]   [6977, 0.45316264488310337]  
13856   [11930, 0.2914800043784019]   [13201, 0.2897033465639696]  
18732     [739, 0.3812160504011599]  [34314, 0.37311532180081447]  
7719   [29303, 0.44100204212811844]  [29306, 0.43850820541229196]  
9797   [25168, 0.30423819912521893]    [9869, 0.3037115795580636]  
3866     [717, 0.30676816437966153]   [8313, 0.28931677571683256]  
4948     [5029, 0.5214020317873379]    [4864, 0.5160719100051497]  
31532   [5976, 0.42579697539360695]  [19065, 0.42205398693849816]

In [41]:
# saving similarity top-k dataframe

df_top_k.to_parquet("../data/movie_top_k_t.parquet")

## Testing

In [203]:
pd.set_option('display.max_colwidth', 50)

In [204]:
query = 'titanic'

df[df["Title"].str.contains("titanic")]

Release Year              Title Origin/Ethnicity  \
6008          1953            titanic         american   
9758          1980  raise the titanic         american   
12857         1996            titanic         american   
13153         1997            titanic         american   
16392         2012         titanic 3d         american   

                      Director  \
6008     directorjeannegulesco   
9758      directorjerryjameson   
12857  directorrobertlieberman   
13153     directorjamescameron   
16392     directorjamescameron   

                                                    Cast  \
6008   actorbarbarastanwyck actorcliftonwebb actorrob...   
9758   actorjasonrobards actoralecguinness actorannea...   
12857  actorgeorgec.scott actorevamariesaint actorpet...   
13153  actorleonardodicaprio actorkatewinslet actorbi...   
16392  actorleonardodicaprio actorkatewinslet actorbi...   

                           Genre  \
6008                   biography   
9758                      action   
12857                  biography   
13153  historical epic, disaster   
16392                      drama   

                                               Wiki Page  \
6008   https://en.wikipedia.org/wiki/titanic_(1953_film)   
9758   https://en.wikipedia.org/wiki/raise_the_titani...   
12857  https://en.wikipedia.org/wiki/titanic_(1996_tv...   
13153  https://en.wikipedia.org/wiki/titanic_(1997_film)   
16392  https://en.wikipedia.org/wiki/titanic_(1997_film)   

                                                    Plot  \
6008   at the last minute, a wealthy american expatri...   
9758   the film opens on the fictional island of svar...   
12857  titanic follows three main story threads.\r\ni...   
13153  in 1996, treasure hunter brock lovett and his ...   
16392  in 1996, treasure hunter brock lovett and his ...   

                                                     doc  \
6008   1953 titanic american directorjeannegulesco ac...   
9758   1980 raise the titanic american directorjerryj...   
12857  1996 titanic american directorrobertlieberman ...   
13153  1997 titanic american directorjamescameron act...   
16392  2012 titanic 3d american directorjamescameron ...   

                                               doc_clean  
6008   1953 titan american directorjeannegulesco acto...  
9758   1980 rais titan american directorjerryjameson ...  
12857  1996 titan american directorrobertlieberman ac...  
13153  1997 titan american directorjamescameron actor...  
16392  2012 titan american directorjamescameron actor...

In [207]:
movie_index = 13153 # Titanic Movie

df_query = df_cos_sim[[movie_index]].sort_values(by=[movie_index]).tail(5)
df_query.shape

(34886, 1)

In [217]:
df_query

13153
6275   0.398865
16004  0.421042
14599  0.472291
16392  0.992128
13153  1.000000

In [223]:
df[df.index == 6275]

Release Year         Title Origin/Ethnicity           Director  \
6275         1955  east of eden         american  directoreliakazan   

                                                   Cast  Genre  \
6275  actorjulieharris actorjamesdean actorraymondma...  drama   

                                              Wiki Page  \
6275  https://en.wikipedia.org/wiki/east_of_eden_(film)   

                                                   Plot  \
6275  the story is set during 1917 and 1918, leading...   

                                                    doc  \
6275  1955 east of eden american directoreliakazan a...   

                                              doc_clean  
6275  1955 east eden american directoreliakazan acto...